In [1]:
import os
import caesar
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import modules as anal
import modules.anal_func as anal_func

from astropy.cosmology import LambdaCDM
import astropy.units as u
from astropy.io import fits
from scipy import interpolate

from modules.powdermod.makesed import MakeSED

cosmo = LambdaCDM(H0 = 68, Om0= 0.3, Ode0= 0.7, Ob0=0.048)
plt.rcParams.update({'font.size': 28})

sb = anal.Simba('cis', size='50_full')

In [14]:
# select the galaxies for the SED

In [2]:
# initialize SED making class
hydro_dir_base = '/mnt/home/glorenzon/simbanator/analize_simba_cgm/output/hdf5/filtered_part_files/'
preselect = '/mnt/home/glorenzon/simbanator/analize_simba_cgm/output/txt/target_selection_for_SED/test_selection.txt'
selection_file = 'selection'
makesed = MakeSED(sb, nnodes=1, model_run_name='m50', hydro_dir_base=hydro_dir_base, selection_file=selection_file, preselect=preselect)

In [21]:
makesed.selection_gals([104], [0]) # create the hdf5 selection
makesed.create_master('cluster') # create the parameters masters

yt : [INFO     ] 2024-09-21 12:09:01,287 Opening /mnt/home/glorenzon/SIMBA_catalogs/simba50_snaps/full//Groups/m50n512_104.hdf5
yt : [INFO     ] 2024-09-21 12:09:02,159 Found 66167 halos
yt : [INFO     ] 2024-09-21 12:09:03,479 Found 4868 galaxies


Inserted aggregated data for: snap104
===================== snap104/code_coods [[24552.39875    32811.19796875 37314.56703125]]
Processing galaxy:  0  in snap  104
Initializing position:  [24552.39875    32811.19796875 37314.56703125]
writing slurm submission master script file
/mnt/home/glorenzon/simbanator/output/hdf5/powderday_sed_out/snap_104/master.snap104.job
===================== snap090/code_coods [[24194.74796875 33212.0446875  37036.18671875]]
Processing galaxy:  0  in snap  90
Initializing position:  [24194.74796875 33212.0446875  37036.18671875]
writing slurm submission master script file
/mnt/home/glorenzon/simbanator/output/hdf5/powderday_sed_out/snap_090/master.snap90.job


In [17]:
fig, ax = makesed.plotsed('090', 0, ret=True)

saving


In [11]:
ax.set_ylim(1e-10, 1e11)
fig.savefig('again.png')

In [22]:
cs = sb.get_caesar(104)

yt : [INFO     ] 2024-09-21 12:11:17,594 Opening /mnt/home/glorenzon/SIMBA_catalogs/simba50_snaps/full//Groups/m50n512_104.hdf5
yt : [INFO     ] 2024-09-21 12:11:17,670 Found 66167 halos
yt : [INFO     ] 2024-09-21 12:11:17,727 Found 4868 galaxies


In [25]:
[i.pos.in_units('code_length').value for i in cs.galaxies][0]

array([24552.39875   , 32811.19796875, 37314.56703125])

In [ ]:
coods_code = np.array([g.pos.in_units('code_length').value for g in cs.galaxies])

In [8]:
path_to_im = '/mnt/home/glorenzon/simbanator/output/hdf5/powderday_sed_out/snap_090/gal_0/convolved.090.hdf5'
import h5py
f = h5py.File(path_to_im, 'r')


In [11]:
convolved_image = f['image_data'][0]
filter_name = f['filter_names'][1]
print(filter_name)

b'g_SDSS.filter'


In [13]:
f['g_SDSS.filter'][...]

array([[0.372     , 0.00526185],
       [0.382     , 0.0152338 ],
       [0.392     , 0.0881389 ],
       [0.402     , 0.191559  ],
       [0.412     , 0.259924  ],
       [0.422     , 0.297598  ],
       [0.432     , 0.327996  ],
       [0.442     , 0.354969  ],
       [0.452     , 0.374454  ],
       [0.462     , 0.399152  ],
       [0.472     , 0.415093  ],
       [0.482     , 0.422689  ],
       [0.492     , 0.438726  ],
       [0.502     , 0.451055  ],
       [0.512     , 0.459661  ],
       [0.522     , 0.452507  ],
       [0.532     , 0.300213  ],
       [0.542     , 0.0467435 ],
       [0.552     , 0.00825236],
       [0.562     , 0.0018465 ],
       [0.572     , 0.00093439]])

In [16]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()
ax = fig.add_subplot(111)

w = f['image_data'].attrs['width']
w_unit = f['image_data'].attrs['width_unit'].astype(str)

cax = ax.imshow(np.log(convolved_image), cmap=plt.cm.viridis,
                origin='lower', extent=[-w, w, -w, w])
ax.tick_params(axis='both', which='major', labelsize=10)
ax.set_xlabel('x ({})'.format(w_unit))
ax.set_ylabel('y ({})'.format(w_unit))
plt.colorbar(cax, label='log Luminosity (ergs/s)', format='%.0e')
plt.title("Convolved image: {}".format(filter_name))
plt.tight_layout()
plt.savefig('first_light.png')
plt.show()

/tmp/ipykernel_12053/3508781549.py:10: RuntimeWarning: divide by zero encountered in log
  cax = ax.imshow(np.log(convolved_image), cmap=plt.cm.viridis,
